In [2]:
# from saxobank.model import streaming
from saxobank.model.chart import charts
from saxobank.model import req
from saxobank.model import common
from aiohttp import client
import json
from saxobank.converter import as_request, from_response
from datetime import datetime
from typing import List

a = charts.ChartSample

now = datetime.now()
mdl = charts.GetResp(Data=[charts.ChartSample(
    CloseAsk=100.0,
    CloseBid=200.0,
    Time='2015-02-02T01:02:03Z')],
    DataVersion=1)
# mdl = charts.GetResp(DataVersion=1.3)
req_mdl = req.GetChartCharts()

d = as_request(mdl.Data)
print(d)
j = json.dumps(d)
print(j)
d2 = json.loads(j)
m = from_response(d2, charts.ChartSample)
print(m)
# m = from_response(d, charts.GetResp)
# print(m)
# print(json.dumps(as_request([mdl])))


[{'CloseAsk': 100.0, 'CloseBid': 200.0, 'Time': '2015-02-02T01:02:03'}]
[{"CloseAsk": 100.0, "CloseBid": 200.0, "Time": "2015-02-02T01:02:03"}]
[ChartSample(CloseAsk=100.0, CloseBid=200.0, Time=datetime.datetime(2015, 2, 2, 1, 2, 3))]


In [1]:
from saxobank.model.chart import charts
from saxobank.converter import as_request, from_response
from dataclasses import fields, asdict
from typing import get_type_hints
from inspect import get_annotations
from json import loads

exp = loads('{"AssetType": "CfdOnIndex", "Uic": 1924, "Horizon": 100, "Mode": "UpTo"}')
mdl = charts.GetReq(**exp)
print(from_response(asdict(mdl), charts.GetReq))

mdl = charts.GetResp(Data=[charts.ChartSample(
    CloseAsk=100.0,
    CloseBid=200.0,
    Time='2015-02-02T01:02:03Z')],
    DataVersion=1)
exp = as_request(mdl.Data)
print(exp)

mdl = charts.GetResp(Data=[{
    "CloseAsk": 100.0,
    "CloseBid": 200.0,
    "Time": '2015-02-02T01:02:03Z'}],
    DataVersion=1)
print(mdl)




/root/git/python-saxobank/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


AttributeError: module 'saxobank.model.chart.charts' has no attribute 'GetReq'

In [1]:
from types import GenericAlias, UnionType
from typing import Union, Any, get_origin, get_args
import inspect
from dataclasses import dataclass, Field, fields, asdict
from saxobank.model.chart import charts
from inspect import get_annotations, getclasstree
from saxobank.model.common import AssetType
from saxobank.model.base import SaxobankModel2
import typing
import types

def check_union_subclass(cls: Union, classinfo: type):
    pass

def checksubclass(cls, classinfo):
    print(f"checking class:{cls} is subclass of: {classinfo}")
    print(f'subclass of type classinfo: {issubclass(classinfo, typing.TypeAlias)}')

    origin = get_origin(cls)
    print(f'origin is: {origin}')

    if origin is not None:
        if origin == Union:
            args = get_args(cls)
            print(f"args are: {args}")
            return all([issubclass(e, classinfo) for e in args])
    else:
        return issubclass(cls, classinfo)


def validate_union(value: Any, schema: Any):
    check = any(validate_field(value, e) for e in typing.get_args(schema))
    print(f'Union checked: {check}')
    return check

def validate_dict(value: Any, schema: Any):
    if not isinstance(value, schema):
        print(f'not dict')
        return False
    
    print(f'check dict')

def validate_list(value: Any, schema: Any):
    if not isinstance(value, list):
        print(f"not list")
        return False
    
    args = typing.get_args(schema)
    if not args:
        print(f"it's list")
        return True
    
    return all(validate_field(e, args[0]) for e in value)


def validate_field(value: Any, schema: Any):
    print(f"checking field:{value} is subclass of: {schema}")
    
    origin = typing.get_origin(schema)
    print(origin)
    if origin:
        if origin == typing.Union:
            return validate_union(value, schema)
        elif issubclass(origin, list):
            return validate_list(value, schema)
        elif issubclass(origin, dict):
            return validate_dict(value, schema)
        print('else')

    else:
        # if isinstance()
        check = isinstance(value, schema)
        print(f"builtin checked: {check}")
        return check


def validate_model(test: SaxobankModel2):
    schema = get_annotations(test.__class__, eval_str=True)
    print(f'schema is: {schema}')

    for (name, value) in asdict(test).items():
        validate_field(value, schema[name])


mdl = charts.GetResp(Data=[charts.ChartSample(
    CloseAsk=100.0,
    CloseBid=200.0,
    Time='2015-02-02T01:02:03Z')],
    DataVersion=3)

# validate_model(mdl)


/home/toshiya/git/python-saxobank/.venv/lib/python3.10/site-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [2]:
from typeguard import check_type
from saxobank.model import streaming
from saxobank.model.base import SaxobankModel2
from inspect import get_annotations
from dataclasses import asdict, fields
from saxobank.model.chart import charts
from saxobank.model.common import ChartRequestMode

def validate_model(test: SaxobankModel2):
    schema = get_annotations(test.__class__, eval_str=True)
    print(f'schema is: {schema}')

    # for (name, value) in asdict(test).items():
    for field in fields(test):
        name = field.name
        value = getattr(test, name)
        print(f'checking.. {value} is instance of {schema[name]}')
        check_type(value, schema[name])
        print('passed')

mdl = charts.ReqSubscriptionsPost(
    Arguments=charts.ChartSubscriptionRequest(
        AssetType=streaming.HeartbeatReason.NoNewData,
        Uic=323,
        Horizon=3,
        # Mode=ChartRequestMode.UpTo,
        Count=3
    )
)

validate_model(mdl)

ValueError: <HeartbeatReason.NoNewData: 'NoNewData'> is not a valid AssetType